CPace sage notebook for generating test vectors for the CPace25519 cipher suite.

The notebook is organized in a series of several subcells.
1.) X25519 definitions and basic string->integer->fieldElement->string conversions back and forth.
2.) Definitions of the Elligator2 primitive (both, one derived from the Hash2Curve code and 
    one with straight-line code from the Elligator paper.
3.) Test vector generation for X25519 and Elligator2 as ported to sage from the ANSI-C code from the
    Endress+Hauser crypto library.
4.) Implementation and test vector generation for the CPace-specific constructions for generator
    calculation and ISK determination.
    
The cells should best be calculated in sequence so that all definitions are available.
Tested with SageMath 8.6 on ubuntu linux.

In [16]:
# 1.) X25519 definitions and tool definitions


#takes an integer and returns a flipped integer
def flipEndianess(u,len):
    bytes = [((u >> (8 * b)) & 255) for b in range(len)]
    return sum([bytes[i] << 8*(len - i - 1) for i in range(len)])
    
def littleEndianByteStringToInteger(k):
    bytes = [(b) for b in k]
    return sum((bytes[i] << (8 * i)) for i in range(len(bytes)))

def clampScalar25519(k):
    bytes = [(b) for b in k]
    bytes[0] &= 248
    bytes[31] &= 127
    bytes[31] |= 64
    length = len(bytes)
    result = sum((bytes[i] << (8 * i)) for i in range(length))
    return Integer(result)

def clampScalarForInversion25519(k):
    bytes = [(b) for b in k]
    bytes[0] &= 248
    #bytes[31] &= 127
    #bytes[31] |= 64
    length = len(bytes)
    result = sum((bytes[i] << (8 * i)) for i in range(length))
    return Integer(result)

def IntegerToLEPrintString(u,numBytes=32):
    u = Integer(u)
    res = ""
    ctr = 0
    while ((u != 0) | (numBytes > 0)):
        byte =  u % 256
        res += ("%02x" % byte)
        u = (u - byte) >> 8
        numBytes = numBytes - 1
        ctr = ctr + 1
        if (ctr % 32) == 0:
            res += "\n"
    return res

def ByteStringToLEPrintString(k):
    bytes = [(b) for b in k]
    res = ""
    ctr = 0
    for x in bytes:
        res += ("%02x" %x)
        ctr = ctr + 1
        if (ctr % 32) == 0:
            res += "\n"        
    return res

def Inverse_X25519(basepoint,scalar_byte_string):
    OrderPrimeSubgroup = 2^252 + 27742317777372353535851937790883648493
    SF = GF(OrderPrimeSubgroup)
    coFactor = SF(8)
    scalar = clampScalar25519(scalar_byte_string)
    inverse_scalar = 1 /  (SF(scalar) * coFactor)
    inverse_scalar_int = Integer(inverse_scalar) * 8
    inverse_scalar_str = integerToLittleEndianString(inverse_scalar_int)
    return X25519(basepoint,inverse_scalar_str,withClamping=0)

def X25519(u_string, scalar_string, withClamping=1):
    prime = 2^255 - 19
    F = GF(prime)
    A = F(486662)
    nonsquare = F(2)   
    E = EllipticCurve(F, [0, A , 0, 1 , 0])
    Twist = EllipticCurve(F, [0, A * nonsquare, 0, 1 * nonsquare^2, 0])

    u = F(littleEndianByteStringToInteger(u_string))     

    if (withClamping == 1):
        scalar = clampScalar25519(scalar_string)
    else:    
        scalar = clampScalarForInversion25519(scalar_string)
    
    d = 1
    v2 = u^3 + A*u^2 + u
    if not v2.is_square():
        print("Input point is on the twist! "),
        E = Twist
        d = nonsquare
        u = d * u
        v2 = u^3 + A*u^2 * nonsquare + u * nonsquare^2
    v = v2.sqrt()
    
    point = E(u, v)
    (resultPoint_u, resultPoint_v, result_Point_z) = point * scalar
    return resultPoint_u / d



In [18]:
#2.) Function definitions for Elligator2
#
# Elligator2 
#
# Naive implementation from the Elligator paper

def elligator2_curve25519(r):
    p = 2^255 - 19
    F = GF(p)
    A = F(486662)
    B = F(1)
    r = F(r)
    
    u = F(2) # Our Non-square
    powerForChi = 2^254 - 10
    
    v = - A / (1 + u * r^2)
    epsilon = (v^3 + A * v^2 + B * v)^powerForChi
    x = epsilon * v - (1 - epsilon) * A/2
    return x

#
# Elligator2 implementation taken from the hash2curve repository on Github
# 

def CMOV(x, y, b):
    """
    Returns x if b=False; otherwise returns y
    """
    return int(not bool(b))*x + int(bool(b))*y

def sgn0_le(x):
    """
    Returns -1 if x is 'negative' (little-endian sense), else 1.
    """
    degree = x.parent().degree()
    if degree == 1:
        # not a field extension
        xi_values = (ZZ(x),)
    else:
        # field extension
        xi_values = ZZR(x)  # extract vector repr of field element (faster than x._vector_())
    sign = 0
    # compute the sign in constant time
    for i in range(0, degree):
        zz_xi = xi_values[i]
        # sign of this digit
        sign_i = CMOV(1, -1, zz_xi % 2 == 1)
        sign_i = CMOV(sign_i, 0, zz_xi == 0)
        # set sign to this digit's sign if sign == 0
        sign = CMOV(sign, sign_i, sign == 0)
    return CMOV(sign, 1, sign == 0)

sgn0 = sgn0_le

def map_to_curve_elligator2_curve25519(u):
    p = 2^255 - 19
    F = GF(p)
    A = F(486662)
    B = F(1)
    
    u = F(u)
    c1 = (p + 3) // 8
    c2 = F(2)^c1
    c3 = sqrt(F(-1))
    c4 = (p - 5) // 8

    tv1 = u^2
    tv1 = 2 * tv1
    xd = tv1 + 1             # Nonzero: -1 is square (mod p), tv1 is not
    x1n = -486662             # x1 = x1n / xd = -486662 / (1 + 2 * u^2)
    tv2 = xd^2
    gxd = tv2 * xd            # gxd = xd^3
    gx1 = 486662 * xd         # 486662 * xd
    gx1 = gx1 + x1n           # x1n + 486662 * xd
    gx1 = gx1 * x1n           # x1n^2 + 486662 * x1n * xd
    gx1 = gx1 + tv2           # x1n^2 + 486662 * x1n * xd + xd^2
    gx1 = gx1 * x1n           # x1n^3 + 486662 * x1n^2 * xd + x1n * xd^2
    tv3 = gxd^2
    tv2 = tv3^2               # gxd^4
    tv3 = tv3 * gxd           # gxd^3
    tv3 = tv3 * gx1           # gx1 * gxd^3
    tv2 = tv2 * tv3           # gx1 * gxd^7
    y11 = tv2^c4              # (gx1 * gxd^7)^((p - 5) / 8)
    y11 = y11 * tv3           # gx1 * gxd^3 * (gx1 * gxd^7)^((p - 5) / 8)
    y12 = y11 * c3
    tv2 = y11^2
    tv2 = tv2 * gxd
    e1 = tv2 == gx1
    y1 = CMOV(y12, y11, e1)  # If g(x1) is square, this is its sqrt
    x2n = x1n * tv1           # x2 = x2n / xd = 2 * u^2 * x1n / xd
    y21 = y11 * u
    y21 = y21 * c2
    y22 = y21 * c3
    gx2 = gx1 * tv1           # g(x2) = gx2 / gxd = 2 * u^2 * g(x1)
    tv2 = y21^2
    tv2 = tv2 * gxd
    e2 = tv2 == gx2
    y2 = CMOV(y22, y21, e2)  # If g(x2) is square, this is its sqrt
    tv2 = y1^2
    tv2 = tv2 * gxd
    e3 = tv2 == gx1
    xn = CMOV(x2n, x1n, e3)  # If e3, x = x1, else x = x2
    y = CMOV(y2, y1, e3)    # If e3, y = y1, else y = y2
    e4 = sgn0(u) == sgn0(y)  # Fix sign of y
    y = CMOV(-y, y, e4)
    return xn/xd


In [20]:
# 3.) Definitions for the X25519 and Elligator2 test cases

class X25519_testCase:
    def __init__(self,u_in, s_in, u_out):
        self.u_in = u_in
        self.s_in = s_in
        self.u_out = u_out

    def runTest(self):
        us = integerToLittleEndianString(self.u_in)
        ss = integerToLittleEndianString(self.s_in)
        u = X25519(us,ss)
        if (u != self.u_out):
            print ("Fail")
            print ("Input u :\n0x%032x\n" % self.u_in)
            print ("Input s :\n0x%032x\n" % self.s_in)
            print ("Correct Result :\n0x%032x\n" % self.u_out)
            print ("Actual Result :\n0x%032x\n" % u)
            return False
        print ("Pass")
        return True
    
    def docOutput(self):
        print ("Test case for X25519:")
        print ("u:"),
        print (IntegerToLEPrintString(self.u_in))
        print ("s:"),
        print (IntegerToLEPrintString(self.s_in))
        print ("r:"),
        print (IntegerToLEPrintString(self.u_out))
        

testCases = []

tv = \
    X25519_testCase(0x4c1cabd0a603a9103b35b326ec2466727c5fb124a4c19435db3030586768dbe6,\
                    0xc49a44ba44226a50185afcc10a4c1462dd5e46824b15163b9d7c52f06be346a5,\
                    0x5285a2775507b454f7711c4903cfec324f088df24dea948e90c6e99d3755dac3)
testCases.append(tv)

tv = X25519_testCase(0x13a415c749d54cfc3e3cc06f10e7db312cae38059d95b7f4d3116878120f21e5,\
                     0xdba18799e16a42cd401eae021641bc1f56a7d959126d25a3c67b4d1d4e9664b,\
                    0x5779ac7a64f7f8e652a19f79685a598bf873b8b45ce4ad7a7d90e87694decb95)
testCases.append(tv)

tv = X25519_testCase(0,\
                     0xc49a44ba44226a50185afcc10a4c1462dd5e46824b15163b9d7c52f06be346a5,\
                     0)
testCases.append(tv)
    
weakp = []
weakp.append(0)
weakp.append(1)
weakp.append(325606250916557431795983626356110631294008115727848805560023387167927233504) #(which has order 8)
weakp.append(39382357235489614581723060781553021112529911719440698176882885853963445705823) #(which also has order 8)
weakp.append(2^255 - 19 - 1)
weakp.append(2^255 - 19)
weakp.append(2^255 - 19 + 1)
weakp.append(2^255 - 19 + 325606250916557431795983626356110631294008115727848805560023387167927233504)
weakp.append(2^255 - 19 + 39382357235489614581723060781553021112529911719440698176882885853963445705823)
weakp.append(2 * (2^255 - 19) - 1)
weakp.append(2 * (2^255 - 19))
weakp.append(2 * (2^255 - 19) + 1)

for x in weakp:
    tv = X25519_testCase (x,0xff9a44ba44226a50185afcc10a4c1462dd5e46824b15163b9d7c52f06be346af,0)
    testCases.append(tv)

for x in testCases:
    x.runTest()

for x in testCases:
    x.docOutput()

    
#
# Elligator 2 test cases
#
#
# Testvector from the NaCl M0 testsuite from E+H
#

etc1_in = \
        0xbc,  0x14,  0x9a,  0x46,  0xd2,  0x93,  0xb0,  0xae, \
        0xea,  0x34,  0x58,  0x13,  0x49,  0xd7,  0x2f,  0x8a, \
        0x5a,  0x96,  0xcd,  0x53,  0x11,  0x02,  0xd6,  0x73, \
        0x79,  0xcd,  0x9b,  0xfa,  0xdd,  0x4e,  0xc8,  0x00

etc1_out = \
        0x66,  0xb6,  0x8f,  0x75,  0x75,  0xcd,  0x28,  0x24, \
        0x03,  0xfc,  0x2b,  0xd3,  0x23,  0xff,  0x04,  0x60, \
        0x12,  0x03,  0xc1,  0xec,  0x55,  0x16,  0xce,  0x24, \
        0x7f,  0x7c,  0x0a,  0xdb,  0xef,  0x05,  0xd3,  0x67

etc2_in = \
        0x89,  0xcf,  0x55,  0xd4,  0xb5,  0xd3,  0xf8,  0x4b, \
        0x16,  0x34,  0x95,  0x7a,  0xc5,  0x03,  0xa3,  0x2b, \
        0x84,  0xba,  0x11,  0x47,  0x1a,  0x96,  0xb2,  0x27, \
        0xbc,  0xa7,  0x0a,  0x0c,  0x3b,  0xf2,  0x63,  0x75

etc2_out = \
        0x1d,  0xb1,  0x63,  0xc8,  0x6c,  0xec,  0xa7,  0x62, \
        0x19,  0x03,  0xc9,  0x41,  0x2d,  0x6d,  0xc7,  0x1b, \
        0x4e,  0xd2,  0x63,  0xb6,  0x87,  0xee,  0xd0,  0x92, \
        0xb1,  0x94,  0xb5,  0xe5,  0x40,  0xbb,  0xa3,  0x08 
    
etc1_in_s = ""
for x in etc1_in:
    etc1_in_s += chr(x)

etc1_out_s = ""
for x in etc1_out:
    etc1_out_s += chr(x)

etc2_in_s = ""
for x in etc2_in:
    etc2_in_s += chr(x)

etc2_out_s = ""
for x in etc2_out:
    etc2_out_s += chr(x)

etc1_in = littleEndianStringToInteger(etc1_in_s)
etc2_in = littleEndianStringToInteger(etc2_in_s)

etc1_out = littleEndianStringToInteger(etc1_out_s)
etc2_out = littleEndianStringToInteger(etc2_out_s)


if (etc1_out == elligator2_curve25519(etc1_in)) & (etc1_out == map_to_curve_elligator2_curve25519(etc1_in)):
    print ("First elligator 2 testvector passed")
print ("tc1 in: 0x%x" % etc1_in)
print ("tc1 out:0x%x" % etc1_out)
print ("Own res:0x%x" % elligator2_curve25519(etc1_in))
print ("H2C res:0x%x" % map_to_curve_elligator2_curve25519(etc1_in))
print ("\n\n")

if (etc2_out == elligator2_curve25519(etc2_in)) & (etc2_out == map_to_curve_elligator2_curve25519(etc2_in)):
    print ("Second elligator 2 testvector passed")
print ("tc2 in: 0x%x" % etc2_in)
print ("tc2 out:0x%x" % etc2_out)
print ("Own res:0x%x" % elligator2_curve25519(etc2_in))
print ("H2C res:0x%x" % map_to_curve_elligator2_curve25519(etc2_in))

print ("\n\n")
print ("Elligator2 test cases")
print ("in: %s" % IntegerToLEString(etc1_in))
print ("out:%s\n" % IntegerToLEString(etc1_out))
print ("in base 10:\n %i" % etc1_in)
print ("out base 10:\n %i\n" % etc1_out)


print ("in: %s" % IntegerToLEString(etc2_in))
print ("out:%s\n\n" % IntegerToLEString(etc2_out))
print ("in base 10:\n %i" % etc2_in)
print ("out base 10:\n %i" % etc2_out)


NameError: name 'integerToLittleEndianByteString' is not defined

In [26]:
# 4.) Definitions for the CPace25519 specific primitives

# We need SHA-512
import hashlib 

def map_to_group_mod_neg_CPace25519(sid, PRS, CI, doPrint = 1, DSI1 = b"CPace25519-1"):
    F = GF(2^255 - 19)
    m = hashlib.sha512()
    H_block_SHA512 = 128
    ZPAD_len = max(0,H_block_SHA512 - len(DSI1) - len(PRS))
    ZPAD = ZPAD_len * "\0"
        
    if doPrint:
        print ("DSI11= %s string (\'%s\') of len(%i)" % (StringToLEString(DSI1),DSI1, len(DSI1)))
        print ("PRS  = %s (\'%s\') string of len(%i)" % (StringToLEString(PRS), PRS, len(PRS)))
        print ("ZPAD = %i zero bytes (before mixing in adversary controlled variable data)" % (len(ZPAD)))
        print ("sid  = %s string of len(%i)" % (StringToLEString(sid), len(sid)))
        print ("CI   = %s\n       (\'%s\') string of len(%i)\n" % (StringToLEString(CI), CI,len(CI)))
    m.update(DSI1)
    m.update(PRS)
    m.update(ZPAD)
    m.update(sid) 
    m.update(CI)
    u = littleEndianStringToInteger(m.digest())
    if doPrint:
        print ("u = SHA512(DSI1||PRS||ZPAD||sid||CI) as 512 bit little-endian int:\n  0x%032x << 256\n+ 0x%32x" \
               % (Integer(u) >> 256, Integer(u) & (2^256 - 1)))
    u = F(u)
    print ("u as reduced base field element coordinate:\n  0x%x" % Integer(u))
    print ("u encoded as little endian byte string:\n    %s" % IntegerToLEString(u))
    u = map_to_curve_elligator2_curve25519(u)
    print ("Elligator2 output G as base field element coordinate:\n  0x%x" % Integer(u))
    print ("Elligator2 output G encoded as little endian byte string:\n    %s" % IntegerToLEString(u,32)),
    return u
    
def KDF_CPace25519(sid,K,Ya,Yb,doPrint = 1):
    DSI2 = b"CPace25519-2"
    m = hashlib.sha512(DSI2)
    m.update(sid) 
    m.update(K)
    m.update(Ya)
    m.update(Yb)
    ISK = m.digest()
    if doPrint:
        print ("DSI2 = %s string (\'%s\') of len(%i)" % (StringToLEString(DSI2),DSI2, len(DSI2)))
        print ("sid  = %s string of len(%i)" % (StringToLEString(sid), len(sid)))
        print ("K    = %s"\
               "       string of len(%i)" % (StringToLEString(K), len(K)))
        print ("Ya   = %s"\
               "       string of len(%i)" % (StringToLEString(Ya), len(Ya)))
        print ("Yb   = %s"\
               "       string of len(%i)" % (StringToLEString(Yb), len(Yb)))
        print ("ISK  = SHA512(DSI2 || sid || K || Ya || Yb)\n" \
               "     = %s" \
               "       string of len(%i)" % (StringToLEString(ISK), len(ISK)))
    return ISK


p = 2^255 - 19
F = GF(p)
A = F(486662)
B = F(1)

sid = hashlib.sha512(b"sid").digest()
sid = sid [:16]
PRS = b"password"
CI = b"AinitiatorBresponderAD"

print ("Derivation of the secret generator G\n")
G = integerToLittleEndianString(map_to_group_mod_neg_CPace25519(sid,PRS,CI,1))

ya = hashlib.sha512(b"ya").digest()
ya = ya [:32]
yb = hashlib.sha512(b"yb").digest()
yb = yb [:32]

Ya = integerToLittleEndianString(X25519(G,ya))
Yb = integerToLittleEndianString(X25519(G,yb))

K1 = X25519(Yb,ya)
K2 = X25519(Ya,yb)

if (K1 != K2):
    print ("Diffie-Hellman did fail!")
else:
    K = integerToLittleEndianString(K1)

    
print ("\nSecret scalar ya=SHA512(\'ya\'), bytes 0...31, as integer:\n  0x%x" % littleEndianStringToInteger(ya))
print ("ya encoded as little endian byte string:\n    %s" % StringToLEString(ya))
print ("Secret scalar yb=SHA512(\'yb\'), bytes 0...31, as integer:\n  0x%x" % littleEndianStringToInteger(yb))
print ("yb encoded as little endian byte string:\n    %s" % StringToLEString(yb))

print ("Public point Ya as integer:\n  0x%x" % littleEndianStringToInteger(Ya))
print ("Ya encoded as little endian byte string:\n    %s" % StringToLEString(Ya))

print ("Public point Yb as integer:\n  0x%x" % littleEndianStringToInteger(Yb))
print ("Yb encoded as little endian byte string:\n    %s" % StringToLEString(Yb))

print ("DH point K as integer:\n  0x%x" % K1)
print ("K encoded as little endian byte string:\n    %s" % StringToLEString(K))

print ("\nDerivation of the intermediate session key ISK:\n")
ISK = KDF_CPace25519(sid,K,Ya,Yb,1)

Derivation of the secret generator G



TypeError: ord() expected string of length 1, but int found

In [141]:
# Test the inverse X25519 operation for strong AuCPace.

# generate deterministic random numbers based on SHA512 and different strings.
# Generate points and scalars from such random inputs.
# Check that the inverse operation is actually inverting X25519.

inStr = "a"
for m in range(100):
    B = integerToLittleEndianString(9)

    s = hashlib.sha512(inStr).digest()
    inStr += "x"
    s = s [:32]

    T = X25519(B,s)
    Ts = integerToLittleEndianString(T)
    print ("T : 0x%x" % littleEndianStringToInteger(Ts))

    r = hashlib.sha512("r" + inStr).digest()
    r = r [:32]
    print ("r : 0x%x" % littleEndianStringToInteger(r))


    U = X25519(Ts,r)
    Us = integerToLittleEndianString(U)
    print ("U : 0x%x" % U)

    IU = Inverse_X25519(Us,r)
    #integerToLittleEndianString(Inverse_X25519(U,r))
    print ("IU : 0x%x" % IU)
    error = IU - T
    if (error):
        print ("Error : 0x%x" % (error))

print ("End")

T : 0x5d7189be6192ffccdb80902ac26c5d38592822a761c7268007200a232a4cd841
r : 0xe8a70b556490ecdbd52c2927464d4eff557807496af234fc496c9f4221bd4423
U : 0x3e6377b3410a60d0ca65190963ad6dce3aeae178aafad369dc2a59c59acc3ceb
IU : 0x5d7189be6192ffccdb80902ac26c5d38592822a761c7268007200a232a4cd841
T : 0x45d7059890c08698d44496d641ccdfd57ba021c789b440b6382aba832fb7a3b1
r : 0xc949e566c0874f3ecb83bb11d8a2467935de2b16b7eaafff159e0157bf9eefb9
U : 0x7fece83b1e3953b43b024ac9f3cdc3e82ea65d060e15afbdb3ed51b12b33ce7e
IU : 0x45d7059890c08698d44496d641ccdfd57ba021c789b440b6382aba832fb7a3b1
T : 0x6a1c902cfa176af32437d097b26938e8c6e6129814dc6d0894da60f6e8a1c769
r : 0xdc9195405fd90b845f0382b79285e65db0d5dd101819e2c3323eb05eb0ecb520
U : 0x5f9d6d9853631e1c0b3f62ca183aca9ddcc8d58c6aa41f9aa6c08256ec7da765
IU : 0x6a1c902cfa176af32437d097b26938e8c6e6129814dc6d0894da60f6e8a1c769
T : 0x73583bc520f4b9e1245e29b92e6b12dc1c8bc0d9b018a1e2626875db2774974
r : 0xe22cb510bd45fc6cbcb929353777925d152c2a9a5b924715d7480aad8b64d447
U : 

T : 0x67a0c2c958558d0c02ec1cc233410dddafb7f4165e03ef8ceab89248d9d2dd6
r : 0xb628744e1ae8a6de0ee249d04edad237b4f9f2971829d3195deaeb332fcaf048
U : 0x6ab2fe8a01e1134a7b0879a3b3c8f9659bae524ad4a8681294ec14aad1eaf51f
IU : 0x67a0c2c958558d0c02ec1cc233410dddafb7f4165e03ef8ceab89248d9d2dd6
T : 0x24e600ad03c24d35d094dbb2fa9e0c5d5042c8fa087132e2cdf7bab2702aa50
r : 0xe50c6877f38cb889f76c025703ee891b2b90fad3ef77a3703c4c9b9146f950e4
U : 0x6ddf6190e15b369706975c1d6e824a4c0a345cf37d8cc39ae788885b3d7cec6a
IU : 0x24e600ad03c24d35d094dbb2fa9e0c5d5042c8fa087132e2cdf7bab2702aa50
T : 0x341b562f185a4a875a2993f7dbec882f8d09d2f455163898f2a229a517dd0400
r : 0x77df91ad46002b99d9540c2e644b9db01581ec3d127ae73a0c1409b1844ffad5
U : 0x442f56ff7dcc7b0ba4fffac94913f4918eb54adb200195fc69bee0e425eb0f82
IU : 0x341b562f185a4a875a2993f7dbec882f8d09d2f455163898f2a229a517dd0400
T : 0x4ad6da73e0b97d25635f1d09d8d649294ece8315eaabe2694134b8c9b4e0897c
r : 0x16c2526a0c38d34790b8d0c27e9e2748bebe5fdfa0996a82204a71f38c68ae90
U : 0x6

IU : 0x49a14f57eb9a83c2c6fba366077284adf5e811b67c7b1c151a842f5e213c90ca
T : 0x4c82c37dab2772618053a343e3bba882d6f2097bdb4b8c50e5e7e6ee7c0812c7
r : 0x97514469f90d90a941ddae90c85dee93fd7650b03637855d9436b39eac738783
U : 0x481f12ab580ade98c2454abf424c32af478cec129c71f289e0fb913d4ecd41af
IU : 0x4c82c37dab2772618053a343e3bba882d6f2097bdb4b8c50e5e7e6ee7c0812c7
T : 0x49829050854de81dcb5f1ba76531d19c549b2403b9cb2de714b60d0f0d02752e
r : 0xbc0477b13eb66c27ea501a1bd24d33615eed2f2f1a22912d37dfa44828d3f5b2
U : 0x7490eb8e151a0342db47d31b406f451cc8a6316fa63e06fc67c346c634257e1e
IU : 0x49829050854de81dcb5f1ba76531d19c549b2403b9cb2de714b60d0f0d02752e
T : 0x69241b1d4ab0e30400d54ec903c500b9895027e8c37db200d1a7c088422276b
r : 0x95c1d51849587d5c649b839a1fd63e1b0214ad409cd5fd07fd9deeab129ead41
U : 0x4266933a62f9aba735ea45874d53a4157485a18546a7b55671a4bfb452ca40f4
IU : 0x69241b1d4ab0e30400d54ec903c500b9895027e8c37db200d1a7c088422276b
T : 0x7a9f7ff6dc0850e45572a9e1d01be1795d3374bf74370d0e70622b80f9d17e6c
r : 

T : 0x76b6d7161b2715385d4d348f5ce26b64e24b648e47a1972b31ef3bfb24ecaea
r : 0xf7e4e4f30f9abb8d309456243bb5d5d3a6f72d22a00c42138a4ee24b88731c75
U : 0x4caa4457c596761972bb121686c5504c1976d0311b0953f27e44eed5ca3ecbde
IU : 0x76b6d7161b2715385d4d348f5ce26b64e24b648e47a1972b31ef3bfb24ecaea
T : 0x1a04bd13d2dfbbffe8f12d54cd6ecab2fa1063b14d7de18485b0e4c461168203
r : 0x27647e281a392e9111a3e6d04d49cc51402e77e7daf93e8ff5b7e1cddd97b4b
U : 0x30a30f6c9ba33fb669783d87fb7af14476f1b1c27543239c70f1da36629685b2
IU : 0x1a04bd13d2dfbbffe8f12d54cd6ecab2fa1063b14d7de18485b0e4c461168203
T : 0x4df66c495af5e5974f0f7a8b89e607f52c969f47f82bb1f93795352034cbe5f6
r : 0x1e204d94cc9c01a40aa6b436e5785141f494fef22fbc0008af3d8489d55c5181
U : 0x4432129cd64621e8ccb27dde3685941c49970b28bd1a3ae3a785cf37afa9910c
IU : 0x4df66c495af5e5974f0f7a8b89e607f52c969f47f82bb1f93795352034cbe5f6
T : 0x559bd88f51b5536968848a0a4b5ce34bd260b26e22972f003878ffe8055c5ff6
r : 0x256c320ae1afb54f198ec84b184726ad6f77f6a2616b73a184ef328504bf4952
U : 0x

In [ ]:
# Definitions for Strong AuCPace




In [163]:

def map_to_group_mod_neg_AuCPace25519(username, password, doPrint = 1, DSI = b"AuCPace25519"):
    F = GF(2^255 - 19)
    m = hashlib.sha512()
    H_block_SHA512 = 128
    ZPAD_len = max(0,H_block_SHA512 - len(DSI) - len(password))
    ZPAD = ZPAD_len * "\0"
        
    if doPrint:
        print ("DSI5 = %s string (\'%s\') of len(%i)" % (StringToLEString(DSI),DSI, len(DSI)))
        print ("pw   = %s (\'%s\') string of len(%i)" % (StringToLEString(password), password, len(password)))
        print ("ZPAD = %i zero bytes (before mixing in adversary controlled variable data)" % (len(ZPAD)))
        print ("name = %s (\'%s\') string of len(%i)" % (StringToLEString(username), username, len(username)))
    m.update(DSI)
    m.update(password)
    m.update(ZPAD)
    m.update(username) 
    u = littleEndianStringToInteger(m.digest())
    if doPrint:
        print ("u = SHA512(DSI||password||ZPAD||username) as 512 bit little-endian int:\n  0x%032x << 256\n+ 0x%32x" \
               % (Integer(u) >> 256, Integer(u) & (2^256 - 1)))
    u = F(u)
    print ("u as reduced base field element coordinate:\n  0x%x" % Integer(u))
    print ("u encoded as little endian byte string:\n    %s" % IntegerToLEString(u))
    u = map_to_curve_elligator2_curve25519(u)
    print ("Elligator2 output Z as base field element coordinate:\n  0x%x" % Integer(u))
    print ("Elligator2 output Z encoded as little endian byte string:\n    %s" % IntegerToLEString(u,32)),
    return u

def MAC_SK_AuCPace25519(ISK,doPrint = 1):
    DSI3 = b"AuCPace25-Ta"
    DSI4 = b"AuCPace25-Tb"
    DSI5 = b"AuCPace25519"
    m = hashlib.sha512(DSI3)
    m.update(ISK) 
    Ta = m.digest()
    Ta = Ta[:16]

    m = hashlib.sha512(DSI4)
    m.update(ISK) 
    Tb = m.digest()
    Tb = Tb[:16]

    m = hashlib.sha512(DSI5)
    m.update(ISK) 
    SK = m.digest()
    if doPrint:
        print ("DSI3 = %s string (\'%s\') of len(%i)" % (StringToLEString(DSI3),DSI3, len(DSI3)))
        print ("DSI4 = %s string (\'%s\') of len(%i)" % (StringToLEString(DSI4),DSI4, len(DSI4)))
        print ("DSI5 = %s string (\'%s\') of len(%i)" % (StringToLEString(DSI5),DSI5, len(DSI5)))
        print ("ISK  = %s string of len(%i)" % (StringToLEString(ISK), len(ISK)))
        print ("Ta  = %s string of len(%i)" % (StringToLEString(Ta), len(Ta)))
        print ("Tb  = %s string of len(%i)" % (StringToLEString(Tb), len(Tb)))
        print ("SK  = %s string of len(%i)" % (StringToLEString(SK), len(SK)))
    return (Ta,Tb,SK)

name = "username"
password = "password"

q = hashlib.sha512("q").digest()
q = q [:32]

r = hashlib.sha512("r").digest()
r = r [:32]

Z = map_to_group_mod_neg_AuCPace25519(name,password)
Zs = integerToLittleEndianString(Z)

print ("\n\n\n")
print ("Z for username, password:\n0x%x" % Z)

print ("q:\n0x%x" % littleEndianStringToInteger(q))


U = X25519(Zs,r)
Us = integerToLittleEndianString(U)
print ("R:\n0x%x" % littleEndianStringToInteger(r))

print ("U:\n0x%x" % U)

UQ = X25519(Us,q)
UQs = integerToLittleEndianString(UQ)

print ("UQ:\n0x%x" % UQ)

ZQ = X25519(Zs,q)
ZQs = integerToLittleEndianString(ZQ)

print ("ZQ directly calculated:\n0x%x" % ZQ)

ZQ2 = Inverse_X25519(UQs,r)
ZQ2s = integerToLittleEndianString(ZQ2)

print ("ZQ as calculated from UQ:\n0x%x" % ZQ2)


import scrypt

w = scrypt.hash(password + name, ZQs, N=1 << 15, r=8, p=1)
w = w[:32]
W = X25519(B,w)

print("Password: \'%s\', length %i" % (password,len(password)))
print("User Name: \'%s\', length %i" % (name,len(name)))
print("Concatenated input to scrypt: \'%s\', length %i" % (password+name,len(password+name)))
print("Salt value Z^q:\n%s" % StringToLEString(ZQs))
print("Password hash w from scrypt with N=1<<15, r=8, p=1\n%s" % StringToLEString(w))

print("Password verifier W = X25519(w)\n0x%x" % X25519(B,w))

ISK = hashlib.sha512("ISK").digest()

MAC_SK_AuCPace25519(ISK)

DSI5 = 417543506163653235353139 string ('AuCPace25519') of len(12)
pw   = 70617373776f7264 ('password') string of len(8)
ZPAD = 108 zero bytes (before mixing in adversary controlled variable data)
name = 757365726e616d65 ('username') string of len(8)
u = SHA512(DSI||password||ZPAD||username) as 512 bit little-endian int:
  0xfec497749d249426e4895e05d0bb4f565aa4423f33d19e6b20aa3837eb77d16e << 256
+ 0xb8f5b1b51c1557c088356d7ca09bd78f259f1d5f4041d466f4edd40f041a0bb3
u as reduced base field element coordinate:
  0xa242d046f835586749962599c699e609a00f2c0f15f584dce322c5bf7e327be
u encoded as little endian byte string:
    be27e3f75b2c32ce4d585ff1c0f2009a609e699c596299748655836f042d240a

Elligator2 output Z as base field element coordinate:
  0x3c01681e4c6d4a43b527c789bcf6046b53ac14c516dfbbbb0f8916826b537f4b
Elligator2 output Z encoded as little endian byte string:
    4b7f536b8216890fbbbbdf16c514ac536b04f6bc89c727b5434a6d4c1e68013c




Z for username, password:
0x3c01681e4c6d4a43b527c789bcf6

('\xbb\xfb\xaeW*\xc1\xef\x86\t\xbe=/p\x0c\xef9',
 '\xde\xd8Ih^\x968\x95\x82X,[\x8e\x0e\x84s',
 'OJ\xee-\x83\x8f\xe6\xddO\x96\xb6\xb6\xe9\x93\xc4?\x87md\x10P\xc8\xf1\xfa\xcd\x8fD!\xc9:\xcfU\x95\xdf\xa9H\xf0\xc6f\x01\xe6T\xfaYb\x06V[\xf4]2FRv\xd2\x12o\xba3J85\xebH')

In [9]:
import hashlib 
m = hashlib.sha512(b"DSI3").digest()
ByteStringToLEPrintString(m)

'f52c2942079a2f2165c956158ff70ccfe5f0821735d045b86c48ad9cc138f96c\n0b307f383c2e7fae696dbe1eb6ef3e86ccde75dc9965ea4c3d8f129285f31c56\n'

In [36]:
StringToLEString((b"hallo"))

TypeError: ord() expected string of length 1, but int found